In [1]:
import haversine
import pandas as pd

In [2]:

TA_level_zips=pd.read_excel("/home/jian/Projects/Big_Lots/New_TA/zips_in_new_ta/BL_Zips in new TA (TA level)_JL_20180330.xlsx",dtype=str)
TA_store=TA_level_zips[['location_id','TA_of_store']].drop_duplicates()
TA_store=TA_store[TA_store['location_id']!="nan"]
TA_store=TA_store.rename(columns={"TA_of_store":"TA"})
TA_store_count=TA_level_zips[['TA_of_zip','Total Store Count in TA (based on zip)']].drop_duplicates()
TA_store_count=TA_store_count[TA_store_count['Total Store Count in TA (based on zip)']!='nan']
TA_store_count=TA_store_count.rename(columns={"TA_of_zip":"TA","Total Store Count in TA (based on zip)":"Store_Count"})
TA_store=pd.merge(TA_store,TA_store_count,on="TA",how="left")
TA_store_1_store=TA_store[TA_store['Store_Count']=="1"]
TA_store_multi_stores=TA_store[TA_store['Store_Count']!="1"]

TA_level_zips=TA_level_zips[['zip_cd','TA_of_zip','revenue_flag']].drop_duplicates()
count_TA_by_zip=TA_level_zips.groupby(['zip_cd'])['TA_of_zip'].count().reset_index()
count_TA_by_zip_1TA=count_TA_by_zip[count_TA_by_zip['TA_of_zip']==1]
count_TA_by_zip_M_TA=count_TA_by_zip[count_TA_by_zip['TA_of_zip']>1]

TA_level_zips_1TA_1_store=TA_level_zips[(TA_level_zips['TA_of_zip'].isin(TA_store_1_store['TA'])) & (TA_level_zips['zip_cd'].isin(count_TA_by_zip_1TA['zip_cd']))]
TA_level_zips_1TA_M_stores=TA_level_zips[(TA_level_zips['TA_of_zip'].isin(TA_store_multi_stores['TA'])) & (TA_level_zips['zip_cd'].isin(count_TA_by_zip_1TA['zip_cd']))]
TA_level_zips_M_TA_M_stores=TA_level_zips[TA_level_zips['zip_cd'].isin(count_TA_by_zip_M_TA['zip_cd'])]

In [3]:
del TA_store_1_store['Store_Count']
TA_level_zips_1TA_1_store=pd.merge(TA_level_zips_1TA_1_store,TA_store_1_store,left_on="TA_of_zip",right_on="TA",how="left")
TA_level_zips_1TA_1_store=TA_level_zips_1TA_1_store[['zip_cd','revenue_flag','location_id']].drop_duplicates()
TA_level_zips_1TA_1_store['location_id']=TA_level_zips_1TA_1_store['location_id'].astype(int)

In [4]:
Multiple_Zip_To_Stores=TA_level_zips_1TA_M_stores.append(TA_level_zips_M_TA_M_stores)
Multiple_Zip_To_Stores=pd.merge(Multiple_Zip_To_Stores,TA_store,left_on="TA_of_zip",right_on="TA",how="left")
del Multiple_Zip_To_Stores['Store_Count']

del Multiple_Zip_To_Stores['TA_of_zip']
del Multiple_Zip_To_Stores['TA']
Multiple_Zip_To_Stores['location_id']=Multiple_Zip_To_Stores['location_id'].astype(int)

In [5]:
Store_level_zip_sales=pd.read_csv("/home/jian/Projects/Big_Lots/New_TA/zips_in_new_ta/sales_by_zip (Store level).csv")
Store_level_zip_sales=Store_level_zip_sales[['net_transaction_amt','zip','revenue_flag','location_id']]
Store_level_zip_sales['zip_cd']=Store_level_zip_sales['zip'].apply(lambda x: str(x).zfill(5))
del Store_level_zip_sales['zip']
Store_level_zip_sales.head(2)
Store_level_zip_sales=Store_level_zip_sales.rename(columns={"revenue_flag":"revenue_flag_Store"})
Store_level_zip_sales=Store_level_zip_sales[Store_level_zip_sales['location_id']!=6990]

In [ ]:
i=1
import datetime
agg_zip_store_in=pd.DataFrame()
agg_zip_store_out=pd.DataFrame()
for zip_cd,group in Multiple_Zip_To_Stores.groupby(['zip_cd']):
    df=pd.merge(group,Store_level_zip_sales,on=["zip_cd","location_id"],how="left")
    df['revenue_flag_Store']=df['revenue_flag_Store'].fillna("XX")
    if "P" in df['revenue_flag_Store'].tolist():
        df_in=df[df['revenue_flag_Store']=="P"]
        if len(df_in)>1:
            df_in=df_in.sort_values(['net_transaction_amt'],ascending=False).reset_index()
            del df_in['index']
            df_in=df_in.head(1)
        agg_zip_store_in=agg_zip_store_in.append(df_in)
    elif "S" in df['revenue_flag_Store'].tolist():
        df_in=df[df['revenue_flag_Store']=="S"]
        if len(df_in)>1:
            df_in=df_in.sort_values(['net_transaction_amt'],ascending=False).reset_index()
            del df_in['index']
            df_in=df_in.head(1)
        agg_zip_store_in=agg_zip_store_in.append(df_in)
    elif "T" in df['revenue_flag_Store'].tolist():
        df_in=df[df['revenue_flag_Store']=="T"]
        if len(df_in)>1:
            df_in=df_in.sort_values(['net_transaction_amt'],ascending=False).reset_index()
            del df_in['index']
            df_in=df_in.head(1)
        agg_zip_store_in=agg_zip_store_in.append(df_in)
    else:
        df_out=df[['zip_cd','revenue_flag','location_id']].drop_duplicates()
        agg_zip_store_out=agg_zip_store_out.append(df_out)

    i=i+1
    if i% 300 ==60:
        print(i, datetime.datetime.now())


In [ ]:
gc.collect()
store_list_1=pd.read_table("/home/jian/BiglotsCode/OtherInput/MediaStormStoreList_Nov15.txt",dtype=str,sep="|")
store_list_2=pd.read_table("/home/jian/BiglotsCode/OtherInput/MediaStormStores_20180703.txt",dtype=str,sep="|")

store_list_1=store_list_1[['location_id','longitude_meas','latitude_meas']]
store_list_2=store_list_2[['location_id','longitude_meas','latitude_meas']]
store_list_1=store_list_1[~store_list_1['location_id'].isin(store_list_2['location_id'])]
store_list=store_list_2.append(store_list_1)
store_list=store_list[store_list['location_id']!="6990"]
store_list=store_list[store_list['location_id']!="145"]

store_list['location_id']=store_list['location_id'].astype(int)
store_list['latitude_meas']=store_list['latitude_meas'].astype(float)
store_list['longitude_meas']=store_list['longitude_meas'].astype(float)

In [ ]:
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/center_of_rentrak_zip.json","r"))

In [ ]:
shortest_out=pd.DataFrame()
agg_zip_store_out['distance']=np.nan
for zip_cd,group in agg_zip_store_out.groupby(['zip_cd']):
    df=pd.merge(group,store_list,on='location_id',how='left')
    df=df.reset_index()
    del df['index']

    for i in range(len(df)):
        zip_center=(df['latitude_meas'][i],df['longitude_meas'][i])
        df['distance'][i]=haversine.haversine(zip_center,zip_centers[zip_cd],miles=True)
        df=df.sort_values(['distance'])
        df_agg=df.head(1)
    shortest_out=shortest_out.append(df_agg)



In [ ]:
agg_zip_store_in['distance']=np.nan
agg_zip_store_in=agg_zip_store_in.reset_index()
del agg_zip_store_in['index']
agg_zip_store_in=pd.merge(agg_zip_store_in,store_list,on='location_id',how='left')
for i in range(len(agg_zip_store_in)):
    zip_cd=agg_zip_store_in['zip_cd'][i]
    store_loc=(agg_zip_store_in['latitude_meas'][i],agg_zip_store_in['longitude_meas'][i])
    try:
        agg_zip_store_in['distance'][i]=haversine.haversine(store_loc,zip_centers[zip_cd],miles=True)
    except:
        agg_zip_store_in['distance'][i]="Zip_Center_Missing"

In [ ]:
TA_level_zips_1TA_1_store['distance']=np.nan
TA_level_zips_1TA_1_store=TA_level_zips_1TA_1_store.reset_index()
del TA_level_zips_1TA_1_store['index']
TA_level_zips_1TA_1_store=pd.merge(TA_level_zips_1TA_1_store,store_list,on='location_id',how='left')
for i in range(len(TA_level_zips_1TA_1_store)):
    zip_cd=TA_level_zips_1TA_1_store['zip_cd'][i]
    store_loc=(TA_level_zips_1TA_1_store['latitude_meas'][i],TA_level_zips_1TA_1_store['longitude_meas'][i])
    try:
        TA_level_zips_1TA_1_store['distance'][i]=haversine.haversine(store_loc,zip_centers[zip_cd],miles=True)
    except:
        TA_level_zips_1TA_1_store['distance'][i]="Zip_Center_Missing"

In [ ]:
zip_store_1_to_1_Part_1=TA_level_zips_1TA_1_store[['zip_cd','revenue_flag','location_id','distance']]
zip_store_1_to_1_Part_2=agg_zip_store_in[['zip_cd','revenue_flag','location_id','distance']]
zip_store_1_to_1_Part_3=shortest_out[['zip_cd','revenue_flag','location_id','distance']]
zip_store_1_to_1=zip_store_1_to_1_Part_1.append(zip_store_1_to_1_Part_2).append(zip_store_1_to_1_Part_3)

In [ ]:
zip_store_1_to_1=zip_store_1_to_1.drop_duplicates()
zip_store_1_to_1.to_csv("/home/jian/BiglotsCode/OtherInput/Zip_in_TA_associated_Store_1_to_1.csv",index=False)